<a href="https://colab.research.google.com/github/dangcaptkd/CS114.K21-/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lấy data từ kaggle!



In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"78c6c8efa90a3136ef03e49e71fb5d9b"}'}

In [0]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 72 May  6 02:33 kaggle.json


In [0]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 109MB/s]


In [0]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


In [0]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
# Loading data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

Xem thử 5 hàng đầu của dataset 

In [0]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


Check the null value

In [0]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


# clean data!

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ, dùng regular expression để loại bỏ các kí tự này

In [0]:
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Gán feature và label 

In [0]:
features = data['headline']
labels = data['is_sarcastic']

Steaming data

In [0]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

Vector hóa các tính năng bằng cách sử dụng TF-IDF Vectorizer (chỗ này để làm gì ta??)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

# Training and testing data

getting training and testing data

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

model 1: Using linear support vector classifier

In [0]:
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print('train:', lsvc.score(features_train, labels_train)) 
print('test:', lsvc.score(features_test, labels_test))   

train: 0.9093524612777362
test: 0.8375748502994012


model 2: Using Gaussuan Naive Bayes

In [0]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('train:', gnb.score(features_train, labels_train))  
print('test:', gnb.score(features_test, labels_test))    

train: 0.7886335868836952
test: 0.7380239520958084


model 3: Logistic Regression

In [0]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('train:', lr.score(features_train, labels_train))   
print('test:', lr.score(features_test, labels_test))     

train: 0.8816458440074094
test: 0.8308383233532934


model 4: Random Forest Classifier

In [0]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('train:', rfc.score(features_train, labels_train)) 
print('test:', rfc.score(features_test, labels_test))    

train: 0.9882946439128207
test: 0.7971556886227545


In [25]:
print('hello world')

hello world
